In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
import datetime
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
df_train = pd.read_csv('train.csv', index_col='id')

In [3]:
df_train.head()

,date,country,store,product,num_sold
id,,,,,
0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [4]:
df_train['date'] = df_train['date'].astype('datetime64')

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   date      136950 non-null  datetime64[ns]
 1   country   136950 non-null  object        
 2   store     136950 non-null  object        
 3   product   136950 non-null  object        
 4   num_sold  136950 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 6.3+ MB


In [6]:
df_train.isnull().sum()

date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

In [7]:
df_train['day'] = df_train['date'].dt.day
df_train['dayofweek'] = df_train['date'].dt.dayofweek 
df_train['week'] = df_train['date'].dt.isocalendar().week
df_train['month'] = df_train['date'].dt.month
df_train['year'] = df_train['date'].dt.year

In [8]:
avr_no_virus = df_train[df_train['year'].isin((2019,2021))].groupby('month')['num_sold'].mean()
avr_virus = df_train[df_train['year']==2020].groupby('month')['num_sold'].mean()

ratio_for_months = avr_no_virus/avr_virus

df_train.loc[df_train['year'] == 2020, 'num_sold'] *= df_train['month'].map(ratio_for_months)

In [9]:
df_train['day_sin'] = np.sin(2 * np.pi * df_train['day']/31)
df_train['day_cos'] = np.cos(2 * np.pi * df_train['day']/31)

df_train['dayofweek_sin'] = np.sin(2 * np.pi * df_train['dayofweek']/6) # indexed [0, 6]
df_train['dayofweek_cos'] = np.cos(2 * np.pi * df_train['dayofweek']/6)

df_train['week_sin'] = np.sin(2 * np.pi * df_train['week']/52)
df_train['week_cos'] = np.cos(2 * np.pi * df_train['week']/52)

df_train['month_sin'] = np.sin(2 * np.pi * df_train['month']/12)
df_train['month_cos'] = np.cos(2 * np.pi * df_train['month']/12)

In [10]:
df_train['is_weekday'] = df_train['dayofweek'].isin([4,5,6]).astype(int)
df_train['is_sunday'] = (df_train['dayofweek']==6).astype(int)

In [11]:
years_list = [2017, 2018, 2019, 2020, 2021, 2022]
holiday_cols = ['AR_holiday', 'CA_holiday', 'EE_holiday', 'JP_holiday', 'ES_holiday']

AR_holidays = holidays.CountryHoliday('AR', years=years_list)
CA_holidays = holidays.CountryHoliday('CA', years=years_list)
EE_holidays = holidays.CountryHoliday('EE', years=years_list)
JP_holidays = holidays.CountryHoliday('JP', years=years_list)
ES_holidays = holidays.CountryHoliday('ES', years=years_list)

# Create Holiday Column 
df_train['AR_holiday'] = df_train.loc[df_train['country'] == 'Argentina', 'date'].apply(lambda d: d in AR_holidays)
df_train['CA_holiday'] = df_train.loc[df_train['country'] == 'Canada', 'date'].apply(lambda d: d in CA_holidays)
df_train['EE_holiday'] = df_train.loc[df_train['country'] == 'Estonia', 'date'].apply(lambda d: d in EE_holidays)
df_train['JP_holiday'] = df_train.loc[df_train['country'] == 'Japan', 'date'].apply(lambda d: d in JP_holidays)
df_train['ES_holiday'] = df_train.loc[df_train['country'] == 'Spain', 'date'].apply(lambda d: d in ES_holidays)

df_train = df_train.fillna('')

df_train['holiday'] = df_train['AR_holiday'].astype('str') + df_train['CA_holiday'].astype('str') \
                + df_train['EE_holiday'].astype('str') + df_train['JP_holiday'].astype('str') \
                + df_train['ES_holiday'].astype('str')

In [12]:


# Create Season Column, Spring = 1, Summer = 2, Autumn = 3, Winter = 4
AR_seasons = np.array([2, 2, 3, 3, 3, 4, 4, 4, 1, 1, 1, 2]).astype(int)
AR_seasons_dict = dict(zip(range(1, 13), AR_seasons))

other_seasons = np.array([4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]).astype(int)
other_seasons_dict = dict(zip(range(1, 13), other_seasons))

df_train['AR_season'] = df_train.loc[df_train['country'] == 'Argentina', 'month'].map(AR_seasons_dict)
df_train['season'] = df_train.loc[df_train['country'] != 'Argentina', 'month'].map(other_seasons_dict)
df_train['season'] = df_train['season'].fillna(df_train['AR_season'])



In [13]:
df_train.drop(columns=['date', 'day', 'dayofweek', 'week', 'month', 'AR_season'] + holiday_cols, inplace=True)

In [14]:
X = df_train.drop(['num_sold'], axis=1)
y = df_train['num_sold']

In [15]:
X = pd.get_dummies(X, columns = ['country', 'store', 'product', 'holiday'])
X.drop('holiday_False', axis = 1, inplace = True)

In [16]:
X

,year,day_sin,day_cos,dayofweek_sin,dayofweek_cos,week_sin,week_cos,month_sin,month_cos,is_weekday,...,country_Spain,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better,holiday_True
id,,,,,,,,,,,,,,,,,,,,,
0,2017,2.012985e-01,0.97953,-2.449294e-16,1.0,0.0,1.0,5.000000e-01,0.866025,1,...,0,0,1,0,1,0,0,0,0,1
1,2017,2.012985e-01,0.97953,-2.449294e-16,1.0,0.0,1.0,5.000000e-01,0.866025,1,...,0,0,1,0,0,1,0,0,0,1
2,2017,2.012985e-01,0.97953,-2.449294e-16,1.0,0.0,1.0,5.000000e-01,0.866025,1,...,0,0,1,0,0,0,1,0,0,1
3,2017,2.012985e-01,0.97953,-2.449294e-16,1.0,0.0,1.0,5.000000e-01,0.866025,1,...,0,0,1,0,0,0,0,1,0,1
4,2017,2.012985e-01,0.97953,-2.449294e-16,1.0,0.0,1.0,5.000000e-01,0.866025,1,...,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136945,2021,-2.449294e-16,1.00000,-8.660254e-01,-0.5,0.0,1.0,-2.449294e-16,1.000000,1,...,1,1,0,0,1,0,0,0,0,0
136946,2021,-2.449294e-16,1.00000,-8.660254e-01,-0.5,0.0,1.0,-2.449294e-16,1.000000,1,...,1,1,0,0,0,1,0,0,0,0
136947,2021,-2.449294e-16,1.00000,-8.660254e-01,-0.5,0.0,1.0,-2.449294e-16,1.000000,1,...,1,1,0,0,0,0,1,0,0,0


In [17]:
scaler = StandardScaler()
scall = scaler.fit_transform(X[['year', 'day_sin', 'day_cos', 'dayofweek_sin', 'dayofweek_cos', 'week_sin', 'week_cos',
                       'month_sin', 'month_cos']])
X_scaler = pd.DataFrame(scall, columns=['year', 'day_sin', 'day_cos', 'dayofweek_sin', 'dayofweek_cos', 'week_sin', 'week_cos',
                       'month_sin', 'month_cos'])

In [18]:
X_scaler

,year,day_sin,day_cos,dayofweek_sin,dayofweek_cos,week_sin,week_cos,month_sin,month_cos
0,-1.414795,0.280207,1.424427,-0.000725,1.154721,-0.000655,1.407389,0.715432,1.225370
1,-1.414795,0.280207,1.424427,-0.000725,1.154721,-0.000655,1.407389,0.715432,1.225370
2,-1.414795,0.280207,1.424427,-0.000725,1.154721,-0.000655,1.407389,0.715432,1.225370
3,-1.414795,0.280207,1.424427,-0.000725,1.154721,-0.000655,1.407389,0.715432,1.225370
4,-1.414795,0.280207,1.424427,-0.000725,1.154721,-0.000655,1.407389,0.715432,1.225370
...,...,...,...,...,...,...,...,...,...
136945,1.414020,-0.001984,1.453646,-1.323872,-0.865580,-0.000655,1.407389,0.006949,1.414478
136946,1.414020,-0.001984,1.453646,-1.323872,-0.865580,-0.000655,1.407389,0.006949,1.414478
136947,1.414020,-0.001984,1.453646,-1.323872,-0.865580,-0.000655,1.407389,0.006949,1.414478
136948,1.414020,-0.001984,1.453646,-1.323872,-0.865580,-0.000655,1.407389,0.006949,1.414478


In [19]:
X_cat = X[['is_weekday', 'is_sunday', 'season', 'country_Argentina', 'country_Canada', 'country_Estonia', 'country_Japan',
  'country_Spain', 'store_Kagglazon', 'store_Kaggle Learn', 'store_Kaggle Store',
       'product_Using LLMs to Improve Your Coding',
       'product_Using LLMs to Train More LLMs',
       'product_Using LLMs to Win Friends and Influence People',
       'product_Using LLMs to Win More Kaggle Competitions',
       'product_Using LLMs to Write Better', 'holiday_True']].astype(float)

In [20]:
X_df = pd.merge(X_scaler, X_cat, left_index=True, right_index=True)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.1, random_state=42)

In [22]:
def smape(act, pred):
    return round(
        np.mean(
            np.abs(pred - act) / 
            ((np.abs(pred) + np.abs(act))/2)
        )*100, 2
    )

In [23]:
def SMAPE(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))



In [24]:
model = XGBRegressor(learning_rate = 0.01, max_depth=8, n_estimators = 250, reg_lambda = 7)

In [25]:
model.fit(X_train, y_train)
predict_test = model.predict(X_test)
predict_train = model.predict(X_train)
print('SMAPE_train: {} | SMAPE_test: {}'.format(SMAPE(y_train, predict_train), smape(y_test, predict_test)))

SMAPE_train: 4.7447704552627705 | SMAPE_test: 4.99


## Validate

In [26]:
df_val = pd.read_csv('test.csv')

df_val['date'] = df_val['date'].astype('datetime64')

df_val['day'] = df_val['date'].dt.day
df_val['dayofweek'] = df_val['date'].dt.dayofweek 
df_val['week'] = df_val['date'].dt.isocalendar().week
df_val['month'] = df_val['date'].dt.month
df_val['year'] = df_val['date'].dt.year

df_val['day_sin'] = np.sin(2 * np.pi * df_val['day']/31)
df_val['day_cos'] = np.cos(2 * np.pi * df_val['day']/31)
df_val['dayofweek_sin'] = np.sin(2 * np.pi * df_val['dayofweek']/6) # indexed [0, 6]
df_val['dayofweek_cos'] = np.cos(2 * np.pi * df_val['dayofweek']/6)
df_val['week_sin'] = np.sin(2 * np.pi * df_val['week']/52)
df_val['week_cos'] = np.cos(2 * np.pi * df_val['week']/52)
df_val['month_sin'] = np.sin(2 * np.pi * df_val['month']/12)
df_val['month_cos'] = np.cos(2 * np.pi * df_val['month']/12)


# Create Holiday Column 
df_val['AR_holiday'] = df_val.loc[df_val['country'] == 'Argentina', 'date'].apply(lambda d: d in AR_holidays)
df_val['CA_holiday'] = df_val.loc[df_val['country'] == 'Canada', 'date'].apply(lambda d: d in CA_holidays)
df_val['EE_holiday'] = df_val.loc[df_val['country'] == 'Estonia', 'date'].apply(lambda d: d in EE_holidays)
df_val['JP_holiday'] = df_val.loc[df_val['country'] == 'Japan', 'date'].apply(lambda d: d in JP_holidays)
df_val['ES_holiday'] = df_val.loc[df_val['country'] == 'Spain', 'date'].apply(lambda d: d in ES_holidays)

df_val = df_val.fillna('')

df_val['holiday'] = df_val['AR_holiday'].astype('str') + df_val['CA_holiday'].astype('str') \
                + df_val['EE_holiday'].astype('str') + df_val['JP_holiday'].astype('str') \
                + df_val['ES_holiday'].astype('str')

df_val['is_weekday'] = df_val['dayofweek'].isin([4,5,6]).astype(int)
df_val['is_sunday'] = (df_val['dayofweek']==6).astype(int)



# Create Season Column, Spring = 1, Summer = 2, Autumn = 3, Winter = 4
AR_seasons = np.array([2, 2, 3, 3, 3, 4, 4, 4, 1, 1, 1, 2]).astype(int)
AR_seasons_dict = dict(zip(range(1, 13), AR_seasons))

other_seasons = np.array([4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4]).astype(int)
other_seasons_dict = dict(zip(range(1, 13), other_seasons))

df_val['AR_season'] = df_val.loc[df_val['country'] == 'Argentina', 'month'].map(AR_seasons_dict)
df_val['season'] = df_val.loc[df_val['country'] != 'Argentina', 'month'].map(other_seasons_dict)
df_val['season'] = df_val['season'].fillna(df_val['AR_season'])



df_val.drop(columns=['date', 'day', 'dayofweek', 'week', 'month', 'AR_season'] + holiday_cols, inplace=True)



In [27]:
X_val = pd.get_dummies(df_val, columns = ['country', 'store', 'product', 'holiday'])
X_val.drop('holiday_False', axis = 1, inplace = True)

In [28]:
scall = scaler.transform(X_val[['year', 'day_sin', 'day_cos', 'dayofweek_sin', 'dayofweek_cos', 'week_sin', 'week_cos',
                       'month_sin', 'month_cos']])
X_val_scal = pd.DataFrame(scall, columns=['year', 'day_sin', 'day_cos', 'dayofweek_sin', 'dayofweek_cos', 'week_sin', 'week_cos',
                       'month_sin', 'month_cos'])

X_val_cat = X_val[['is_weekday', 'is_sunday', 'season', 'country_Argentina', 'country_Canada', 'country_Estonia', 'country_Japan',
  'country_Spain', 'store_Kagglazon', 'store_Kaggle Learn', 'store_Kaggle Store',
       'product_Using LLMs to Improve Your Coding',
       'product_Using LLMs to Train More LLMs',
       'product_Using LLMs to Win Friends and Influence People',
       'product_Using LLMs to Win More Kaggle Competitions',
       'product_Using LLMs to Write Better', 'holiday_True']].astype(float)

X_validate = pd.merge(X_val_scal, X_val_cat, left_index=True, right_index=True)

In [29]:
X_validate

,year,day_sin,day_cos,dayofweek_sin,dayofweek_cos,week_sin,week_cos,month_sin,month_cos,is_weekday,...,country_Spain,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better,holiday_True
0,2.121224,0.280207,1.424427,-1.323872,0.481287,-0.000655,1.407389,0.715432,1.225370,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2.121224,0.280207,1.424427,-1.323872,0.481287,-0.000655,1.407389,0.715432,1.225370,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,2.121224,0.280207,1.424427,-1.323872,0.481287,-0.000655,1.407389,0.715432,1.225370,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,2.121224,0.280207,1.424427,-1.323872,0.481287,-0.000655,1.407389,0.715432,1.225370,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,2.121224,0.280207,1.424427,-1.323872,0.481287,-0.000655,1.407389,0.715432,1.225370,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27370,2.121224,-0.001984,1.453646,-1.323872,0.481287,-0.000655,1.407389,0.006949,1.414478,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
27371,2.121224,-0.001984,1.453646,-1.323872,0.481287,-0.000655,1.407389,0.006949,1.414478,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
27372,2.121224,-0.001984,1.453646,-1.323872,0.481287,-0.000655,1.407389,0.006949,1.414478,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
27373,2.121224,-0.001984,1.453646,-1.323872,0.481287,-0.000655,1.407389,0.006949,1.414478,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [32]:
pred_val = model.predict(X_validate)

In [34]:
np.round(pred_val)

array([ 41.,  42.,   6., ..., 120., 678., 555.], dtype=float32)

In [87]:
pred_val

array([ 40.99251  ,  41.720802 ,   6.3141837, ..., 119.59037  ,
       678.36694  , 555.12646  ], dtype=float32)

In [93]:
test_country = df_val[['id', 'country']].copy()
test_country['num_sold']=np.round(pred_val)

In [105]:
mean_by_country = test_country.groupby('country')['num_sold'].mean()
sum_mean_country = mean_by_country.sum()
ratios = (sum_mean_country*0.31)/mean_by_country
ratios

country
Argentina    4.662925
Canada       0.957403
Estonia      1.783106
Japan        1.246342
Spain        1.656480
Name: num_sold, dtype: float32

In [107]:
predictions_111 = test_country.apply(lambda x: x['num_sold'] * ratios.loc[x['country']], axis=1)
predictions_111.head()

0    191.179915
1    195.842840
2     27.977549
3    177.191141
4    144.550668
dtype: float64

In [108]:
predictions_111

0         191.179915
1         195.842840
2          27.977549
3         177.191141
4         144.550668
            ...     
27370    1227.451558
27371    1189.352522
27372     198.777580
27373    1123.093328
27374     919.346309
Length: 27375, dtype: float64

In [111]:
submission_9 = pd.DataFrame({'id':id_val, 'num_sold': np.round(predictions_111)})
submission_9.to_csv('submission_9.csv', index=False)

In [112]:
pd.read_csv("submission_9.csv")

,id,num_sold
0,136950,191.0
1,136951,196.0
2,136952,28.0
3,136953,177.0
4,136954,145.0
...,...,...
27370,164320,1227.0
27371,164321,1189.0
27372,164322,199.0
27373,164323,1123.0


In [76]:
num_sold = round(num_sold * case_when(
            country == 'Argentina' ~ 3.372,
            country == 'Spain' ~ 1.600,
            country == 'Japan' ~ 1.394,
            country == 'Estonia' ~ 1.651,
            country == 'Canada' ~ 0.850)))

SyntaxError: unmatched ')' (1137137491.py, line 6)

In [116]:
old_df = pd.read_csv('submission_3.csv')
old_df['num_sold']

0         46
1         67
2         16
3         48
4         42
        ... 
27370    716
27371    690
27372    110
27373    638
27374    520
Name: num_sold, Length: 27375, dtype: int64

In [117]:
test_country = df_val[['id', 'country']].copy()
test_country['num_sold']=old_df['num_sold']

In [118]:
test_country

,id,country,num_sold
0,136950,Argentina,46
1,136951,Argentina,67
2,136952,Argentina,16
3,136953,Argentina,48
4,136954,Argentina,42
...,...,...,...
27370,164320,Spain,716
27371,164321,Spain,690
27372,164322,Spain,110
27373,164323,Spain,638


In [119]:
predictions_1111 = test_country.apply(lambda x: x['num_sold'] * ratios.loc[x['country']], axis=1)
predictions_1111.head()

0    214.494539
1    312.415959
2     74.606796
3    223.820389
4    195.842840
dtype: float64

In [120]:
predictions_1111

0         214.494539
1         312.415959
2          74.606796
3         223.820389
4         195.842840
            ...     
27370    1186.039562
27371    1142.971087
27372     182.212782
27373    1056.834135
27374     861.369514
Length: 27375, dtype: float64

In [123]:
submission_10 = pd.DataFrame({'id':id_val, 'num_sold': np.round(predictions_1111)})
submission_10.to_csv("submission_10.csv", index = False)